## GAN

In [20]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.initializers import random_normal

import matplotlib.pyplot as plt

import sys

import numpy as np

In [27]:
class GAN():
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, kernel_initializer=random_normal(), input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(1024, kernel_initializer=random_normal()))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones(batch_size)
        fake = np.zeros(batch_size)

        for epoch in range(epochs+1):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()

In [28]:
if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=500, batch_size=128, sample_interval=50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 1024)              3146752   
_________________________________________________________________
leaky_re_lu_23 (LeakyReLU)   (None, 1024)              0         
_________________________________________________________________
dense_32 (Dense)             (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_24 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_25 (LeakyReLU)   (None, 256)               0         
__________

C:\Users\Jwp\Anaconda3\envs\py_ml\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.916724, acc.: 14.06%] [G loss: 0.203265]


C:\Users\Jwp\Anaconda3\envs\py_ml\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 1.407026, acc.: 49.22%] [G loss: 0.310134]
2 [D loss: 0.767518, acc.: 52.34%] [G loss: 0.976858]
3 [D loss: 0.536814, acc.: 71.48%] [G loss: 1.482298]
4 [D loss: 0.608999, acc.: 68.36%] [G loss: 1.425764]
5 [D loss: 0.603254, acc.: 67.58%] [G loss: 1.261951]
6 [D loss: 0.608104, acc.: 62.89%] [G loss: 1.346227]
7 [D loss: 0.645388, acc.: 64.45%] [G loss: 1.454173]
8 [D loss: 0.503899, acc.: 79.30%] [G loss: 1.730278]
9 [D loss: 0.600368, acc.: 76.95%] [G loss: 1.653968]
10 [D loss: 0.494538, acc.: 77.73%] [G loss: 1.638447]
11 [D loss: 0.545364, acc.: 73.83%] [G loss: 1.862750]
12 [D loss: 0.435966, acc.: 84.77%] [G loss: 1.904807]
13 [D loss: 0.492829, acc.: 81.25%] [G loss: 1.897865]
14 [D loss: 0.406598, acc.: 88.67%] [G loss: 1.999434]
15 [D loss: 0.416430, acc.: 85.55%] [G loss: 2.215102]
16 [D loss: 0.517646, acc.: 80.86%] [G loss: 2.023910]
17 [D loss: 0.415574, acc.: 84.77%] [G loss: 2.111092]
18 [D loss: 0.413358, acc.: 86.72%] [G loss: 2.388320]
19 [D loss: 0.48236

150 [D loss: 0.742470, acc.: 52.34%] [G loss: 1.304355]
151 [D loss: 0.756858, acc.: 46.88%] [G loss: 1.334283]
152 [D loss: 0.765073, acc.: 50.39%] [G loss: 1.285250]
153 [D loss: 0.689976, acc.: 56.64%] [G loss: 1.382980]
154 [D loss: 0.699459, acc.: 57.81%] [G loss: 1.334750]
155 [D loss: 0.708367, acc.: 58.20%] [G loss: 1.329101]
156 [D loss: 0.675999, acc.: 59.77%] [G loss: 1.328014]
157 [D loss: 0.686112, acc.: 60.94%] [G loss: 1.293118]
158 [D loss: 0.745829, acc.: 58.98%] [G loss: 1.322809]
159 [D loss: 0.696624, acc.: 52.34%] [G loss: 1.402045]
160 [D loss: 0.722841, acc.: 57.03%] [G loss: 1.324733]
161 [D loss: 0.709889, acc.: 56.64%] [G loss: 1.343920]
162 [D loss: 0.717297, acc.: 55.86%] [G loss: 1.420368]
163 [D loss: 0.718440, acc.: 59.38%] [G loss: 1.282129]
164 [D loss: 0.696276, acc.: 57.03%] [G loss: 1.366764]
165 [D loss: 0.689060, acc.: 58.98%] [G loss: 1.279398]
166 [D loss: 0.670574, acc.: 58.98%] [G loss: 1.339247]
167 [D loss: 0.665053, acc.: 62.50%] [G loss: 1.

297 [D loss: 0.762861, acc.: 46.09%] [G loss: 1.100984]
298 [D loss: 0.738942, acc.: 50.78%] [G loss: 1.150349]
299 [D loss: 0.731233, acc.: 53.91%] [G loss: 1.123384]
300 [D loss: 0.749451, acc.: 50.78%] [G loss: 1.124364]
301 [D loss: 0.715565, acc.: 49.61%] [G loss: 1.123026]
302 [D loss: 0.714660, acc.: 55.86%] [G loss: 1.100856]
303 [D loss: 0.712630, acc.: 55.86%] [G loss: 1.157034]
304 [D loss: 0.705230, acc.: 53.91%] [G loss: 1.175176]
305 [D loss: 0.713973, acc.: 54.30%] [G loss: 1.181544]
306 [D loss: 0.695547, acc.: 54.69%] [G loss: 1.196556]
307 [D loss: 0.697733, acc.: 55.47%] [G loss: 1.201265]
308 [D loss: 0.727984, acc.: 48.83%] [G loss: 1.140935]
309 [D loss: 0.735326, acc.: 47.66%] [G loss: 1.197616]
310 [D loss: 0.739246, acc.: 53.12%] [G loss: 1.158846]
311 [D loss: 0.736875, acc.: 46.88%] [G loss: 1.106818]
312 [D loss: 0.726989, acc.: 52.34%] [G loss: 1.130731]
313 [D loss: 0.658519, acc.: 62.89%] [G loss: 1.126569]
314 [D loss: 0.700001, acc.: 55.08%] [G loss: 1.

444 [D loss: 0.682482, acc.: 58.59%] [G loss: 1.096239]
445 [D loss: 0.677552, acc.: 55.08%] [G loss: 1.084449]
446 [D loss: 0.691605, acc.: 55.08%] [G loss: 1.116174]
447 [D loss: 0.691627, acc.: 56.64%] [G loss: 1.093160]
448 [D loss: 0.688444, acc.: 53.12%] [G loss: 1.100723]
449 [D loss: 0.662089, acc.: 59.77%] [G loss: 1.020172]
450 [D loss: 0.692407, acc.: 52.34%] [G loss: 1.041900]
451 [D loss: 0.681752, acc.: 56.64%] [G loss: 1.053547]
452 [D loss: 0.694741, acc.: 50.78%] [G loss: 1.075951]
453 [D loss: 0.696651, acc.: 51.56%] [G loss: 1.091083]
454 [D loss: 0.711942, acc.: 53.52%] [G loss: 1.069325]
455 [D loss: 0.693960, acc.: 52.73%] [G loss: 1.072907]
456 [D loss: 0.688140, acc.: 55.86%] [G loss: 1.122003]
457 [D loss: 0.703298, acc.: 51.17%] [G loss: 1.054364]
458 [D loss: 0.693801, acc.: 50.39%] [G loss: 1.057025]
459 [D loss: 0.687802, acc.: 55.08%] [G loss: 1.076481]
460 [D loss: 0.711338, acc.: 48.83%] [G loss: 1.088633]
461 [D loss: 0.678004, acc.: 55.47%] [G loss: 1.